### Activity Analysis

In [1]:
import pandas as pd

In [4]:
# Load the rawdata
raw_df = pd.read_excel("rawdata.xlsx")

In [5]:
raw_df.head()

,date,time,sensor,location,number,activity,position,location.1
0,2024-01-16,10:10:30,0.0,A1,1,placed,inside,A1
1,2024-01-16,10:12:30,1.0,A2,1,picked,inside,A2
2,2024-01-16,10:28:30,1.0,A3,1,picked,Inside,A3
3,2024-01-16,10:38:20,0.0,A4,1,placed,Inside,A4
4,2024-01-16,10:45:20,1.0,A5,1,placed,inside,A5


In [19]:
import pandas as pd

def create_output(raw_df):
    date_format = "%Y-%m-%d"
    time_format = "%H:%M:%S"  

    # To convert 'date' column to datetime
    raw_df['date'] = pd.to_datetime(raw_df['date'], format=date_format)

    # To check if 'time' column exists and convert it to datetime.time
    if "time" in raw_df.columns:
        raw_df['time'] = pd.to_datetime(raw_df['time'], format=time_format).dt.time

        # Combine 'date' and 'time' into a 'datetime' column
        raw_df['datetime'] = pd.to_datetime(
            raw_df['date'].astype(str) + ' ' + raw_df['time'].astype(str))
    else:
        raw_df['datetime'] = raw_df['date']

    raw_df.dropna(inplace=True)

    # To alculate the datewise total duration for each inside and outside position
    duration_inside = (raw_df[raw_df['position'] == 'inside']
                       .groupby(pd.Grouper(key='date'))
                       ['datetime'].apply(lambda x: x.max() - x.min()).groupby(level=0).sum())

    duration_outside = (raw_df[raw_df['position'] != 'inside']
                        .groupby(pd.Grouper(key='date'))
                        ['datetime'].apply(lambda x: x.max() - x.min() if not x.empty else pd.Timedelta(0)).groupby(level=0).sum())

    # To calculate the datewise number of picking and placing activities
    picking_count = raw_df[raw_df['activity'] == 'picked'].groupby('date').size()
    placing_count = raw_df[raw_df['activity'] == 'placed'].groupby('date').size()

    # Now combine the results into a single dataframe
    output = pd.DataFrame({
        'date': duration_inside.index,
        'duration_inside': duration_inside.dt.total_seconds() / 3600,  
        'duration_outside': duration_outside.dt.total_seconds() / 3600,  
        'picking_count': picking_count,
        'placing_count': placing_count
    })

    return output


# Generate the output
output = create_output(raw_df)

# Display the output dataframe
print(output)


                 date  duration_inside  duration_outside  picking_count  \
date                                                                      
2024-01-16 2024-01-16        11.950000          0.163889             40   
2024-01-17 2024-01-17         3.180556               NaN             10   
2024-01-18 2024-01-18         9.330556          8.475000             37   

            placing_count  
date                       
2024-01-16             40  
2024-01-17              9  
2024-01-18             38  


In [21]:
import pandas as pd

# Create a multi-level index
index = pd.MultiIndex.from_product([['A', 'B'], [1, 2], [3, 4]], names=['level1', 'level2', 'level3'])

# Create a DataFrame with the multi-level index
df = pd.DataFrame({'value': [1, 2, 3, 4, 5, 6, 7, 8]}, index=index)

# # Group by the first level (level=0)
# grouped = df.groupby(level=0)

# # Perform an operation on each group
# result = grouped.sum()
# print(result)

In [22]:
df.head()

value
level1 level2 level3       
A      1      3           1
              4           2
       2      3           3
              4           4
B      1      3           5